In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import skfuzzy as fuzz

df = pd.read_csv("covid_neighbors.csv")
print("Dataset:")
display(df)
X = df[["Total_Cases", "Total_Deaths", "Population"]]

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

print("Normalized Data:")
display(pd.DataFrame(X_scaled, columns=X.columns))


X_fcm = X_scaled.T  # Transpose required

cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    X_fcm,
    c=2,
    m=2,
    error=0.005,
    maxiter=1000,
    init=None
)

membership = pd.DataFrame(
    u.T,
    columns=["Cluster 1 Membership", "Cluster 2 Membership"]
)
membership["Country"] = df["Country"]

print("FCM Membership Values:")
display(membership)

membership["Final Cluster (FCM)"] = membership[
    ["Cluster 1 Membership", "Cluster 2 Membership"]
].idxmax(axis=1)

print("Final FCM Clusters:")
display(membership)

print("Fuzzy Partition Coefficient (FPC):", fpc)

kmeans = KMeans(n_clusters=2, random_state=42)
df["KMeans Cluster"] = kmeans.fit_predict(X_scaled)

print("K-Means Clustering Result:")
display(df)

comparison = pd.DataFrame({
    "Country": df["Country"],
    "FCM Cluster": membership["Final Cluster (FCM)"],
    "KMeans Cluster": df["KMeans Cluster"]
})

print("FCM vs K-Means Comparison:")
display(comparison)



Dataset:


,Country,Total_Cases,Total_Deaths,Population
0,Pakistan,274289,5842,220892331
1,India,1480073,33408,1380004385
2,China,86783,4656,1439323776
3,Iran,293606,15912,83992949
4,Afghanistan,36263,1269,38928346


Normalized Data:


,Total_Cases,Total_Deaths,Population
0,0.164860,0.142288,0.129938
1,1.000000,1.000000,0.957641
2,0.034991,0.105386,1.000000
3,0.178239,0.455615,0.032180
4,0.000000,0.000000,0.000000


FCM Membership Values:


...

,Cluster 1 Membership,Cluster 2 Membership,Country,Final Cluster (FCM)
0,0.995312,0.004688,Pakistan,Cluster 1 Membership
1,0.008433,0.991567,India,Cluster 2 Membership
2,0.676247,0.323753,China,Cluster 1 Membership
3,0.937779,0.062221,Iran,Cluster 1 Membership
4,0.970364,0.029636,Afghanistan,Cluster 1 Membership


Fuzzy Partition Coefficient (FPC): 0.8723711825142809
K-Means Clustering Result:


,Country,Total_Cases,Total_Deaths,Population,KMeans Cluster
0,Pakistan,274289,5842,220892331,1
1,India,1480073,33408,1380004385,0
2,China,86783,4656,1439323776,1
3,Iran,293606,15912,83992949,1
4,Afghanistan,36263,1269,38928346,1


FCM vs K-Means Comparison:


,Country,FCM Cluster,KMeans Cluster
0,Pakistan,Cluster 1 Membership,1
1,India,Cluster 2 Membership,0
2,China,Cluster 1 Membership,1
3,Iran,Cluster 1 Membership,1
4,Afghanistan,Cluster 1 Membership,1
